In [7]:
import numpy as np
import pandas as pd

In [68]:
output_folder = '../output/mlsurrogate/'
# jobs = 20000
job_sim_file = '/home/sairam/ATLASGRIDV2/ATLAS-GRID-SIMULATION/output/mlsurrogate/ml_surroagte_net2_JOBS.csv'
state_sim_file = '/home/sairam/ATLASGRIDV2/ATLAS-GRID-SIMULATION/output/mlsurrogate/ml_surroagte_net2_STATE.csv'
jobs_df = pd.read_csv(job_sim_file)
states_df = pd.read_csv(state_sim_file)

In [69]:
print(jobs_df.head())

       JOB_ID          SITE    CPU    STATUS  MEMORY  CORES         FLOPS  \
0  6088783954  NET2_Amherst  cpu-0  finished     0.0      1  8.731108e+10   
1  6088783958  NET2_Amherst  cpu-0  finished     0.0      1  8.597622e+10   
2  6088783961  NET2_Amherst  cpu-0  finished     0.0      1  8.637807e+10   
3  6088783912  NET2_Amherst  cpu-0  finished     0.0      1  8.762557e+10   
4  6088783944  NET2_Amherst  cpu-0  finished     0.0      1  8.996333e+10   

   EXECUTION_TIME  IO_SIZE  IO_TIME  CPU_CONSUMPTION_TIME    CREATION_TIME  \
0       44774.912      0.0      0.0               49972.0  1/26/2024 17:34   
1       44090.368      0.0      0.0               49208.0  1/26/2024 17:34   
2       44296.448      0.0      0.0               49438.0  1/26/2024 17:34   
3       44936.192      0.0      0.0               50152.0  1/26/2024 17:34   
4       46135.040      0.0      0.0               51490.0  1/26/2024 17:34   

            START_TIME             END_TIME  QUEUE_TIME  
0  2024-01

Verifying the Jobs dataset


In [70]:
# Check if in the jobs dataframe, each job has a creation time less than or equal to end time 
for col in ["CREATION_TIME", "START_TIME", "END_TIME"]:
    jobs_df[col] = pd.to_datetime(jobs_df[col], errors="coerce", infer_datetime_format=True)
    
jobs_df["valid_creation"] = jobs_df["CREATION_TIME"] < jobs_df["START_TIME"]
jobs_df["valid_order"] = jobs_df["START_TIME"] <= jobs_df["END_TIME"]
jobs_df["is_valid"] = jobs_df["valid_creation"] & jobs_df["valid_order"]
print(jobs_df[jobs_df["is_valid"] == False])

          JOB_ID          SITE     CPU    STATUS  MEMORY  CORES         FLOPS  \
0     6088783954  NET2_Amherst   cpu-0  finished     0.0      1  8.731108e+10   
1     6088783958  NET2_Amherst   cpu-0  finished     0.0      1  8.597622e+10   
2     6088783961  NET2_Amherst   cpu-0  finished     0.0      1  8.637807e+10   
3     6088783912  NET2_Amherst   cpu-0  finished     0.0      1  8.762557e+10   
4     6088783944  NET2_Amherst   cpu-0  finished     0.0      1  8.996333e+10   
...          ...           ...     ...       ...     ...    ...           ...   
2963  6084903914  NET2_Amherst  cpu-84  finished     0.0      1  5.416320e+07   
2964  6084842925  NET2_Amherst  cpu-84  finished     0.0      1  1.207315e+09   
2965  6084842926  NET2_Amherst  cpu-84  finished     0.0      1  1.174118e+09   
2966  6084842931  NET2_Amherst  cpu-84  finished     0.0      1  1.179360e+09   
2967  6084842932  NET2_Amherst  cpu-84  finished     0.0      1  1.389024e+09   

      EXECUTION_TIME  IO_SI

/tmp/ipykernel_6926/1069307835.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  jobs_df[col] = pd.to_datetime(jobs_df[col], errors="coerce", infer_datetime_format=True)
/tmp/ipykernel_6926/1069307835.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  jobs_df[col] = pd.to_datetime(jobs_df[col], errors="coerce", infer_datetime_format=True)
/tmp/ipykernel_6926/1069307835.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pde

In [71]:
print(states_df.head())

   ID      JOB_ID CPU_NAME     STATE            TIMESTAMP          SITE  \
0   1  6088783954    cpu-0  assigned  2024-01-22 12:47:00  NET2_Amherst   
1   2  6088783954    cpu-0   running  2024-01-22 12:47:00  NET2_Amherst   
2   3  6088783958    cpu-0  assigned  2024-01-22 12:47:00  NET2_Amherst   
3   4  6088783961    cpu-0  assigned  2024-01-22 12:47:00  NET2_Amherst   
4   5  6088783912    cpu-0  assigned  2024-01-22 12:47:00  NET2_Amherst   

   AVAILABLE_SITE_CORES  AVAILABLE_SITE_CPUS      WORKLOAD  NINPUT_FILES  \
0                  3488                  109  8.731108e+10             0   
1                  3488                  109  8.731108e+10             0   
2                  3487                  109  8.597622e+10             0   
3                  3486                  109  8.637807e+10             0   
4                  3485                  109  8.762557e+10             0   

   NOUTPUT_FILES  INPUT_FILE_BYTES  OUTPUT_FILE_BYTES  SITE_PENDING_JOBS  \
0              3

Verifying the States dataframe

In [72]:
# states_df = states_df.sort_values(by="TIMESTAMP")
states_df["TIMESTAMP"] = pd.to_datetime(states_df["TIMESTAMP"], errors="coerce", infer_datetime_format=True)
states_df["TIME_DELTA"] = states_df["TIMESTAMP"].diff().dt.total_seconds().fillna(0).astype(int)

/tmp/ipykernel_6926/3549557007.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  states_df["TIMESTAMP"] = pd.to_datetime(states_df["TIMESTAMP"], errors="coerce", infer_datetime_format=True)


In [73]:
states_df["valid"] = states_df["TIME_DELTA"] >= 0
print(states_df[states_df["valid"] == False])
states_df.to_csv(output_folder + 'z_processed_states.csv', index=False)

Empty DataFrame
Columns: [ID, JOB_ID, CPU_NAME, STATE, TIMESTAMP, SITE, AVAILABLE_SITE_CORES, AVAILABLE_SITE_CPUS, WORKLOAD, NINPUT_FILES, NOUTPUT_FILES, INPUT_FILE_BYTES, OUTPUT_FILE_BYTES, SITE_PENDING_JOBS, SITE_ASSIGNED_JOBS, SITE_FINISHED_JOBS, SITE_FAILED_JOBS, TIME_DELTA, valid]
Index: []


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from torchsummary import summary

# --- 1. Define Model Parameters based on your data dimensions ---
SEQUENCE_LENGTH = 10
NUM_FEATURES = 17

# These must match your actual label vocab sizes from preprocessing:
# NUM_JOBS = np.unique(y_job_id_train_real).size
# NUM_STATES = np.unique(y_state_train_real).size
NUM_JOBS = 4000   # TODO: replace with actual unique JOB_ID count
NUM_STATES = 4    # TODO: replace with actual unique STATE count

# Hyperparameters for the Transformer
D_MODEL = 64
NUM_HEADS = 4
NUM_LAYERS = 2
FF_DIM = 128
DROPOUT_RATE = 0.1
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

# Data split sizes (given)
TOTAL_SEQUENCES = 12529
TRAIN_SAMPLES = 10023
TEST_SAMPLES = TOTAL_SEQUENCES - TRAIN_SAMPLES  # 2506

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 2. Positional Encoding ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

# --- 3. Multi-Task Transformer Model ---
class GridSurrogate(nn.Module):
    def __init__(self, num_features, d_model, num_heads, num_layers, ff_dim, num_jobs, num_states, dropout_rate=0.1):
        super(GridSurrogate, self).__init__()
        self.input_projection = nn.Linear(num_features, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=num_heads, dim_feedforward=ff_dim,
            dropout=dropout_rate, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.job_id_head = nn.Linear(d_model, num_jobs)
        self.state_head = nn.Linear(d_model, num_states)
        self.time_delta_head = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_projection(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x_pooled = self.avg_pool(x.transpose(1, 2)).squeeze(2)
        job_id_pred = self.job_id_head(x_pooled)
        state_pred = self.state_head(x_pooled)
        time_delta_pred = self.time_delta_head(x_pooled)
        return job_id_pred, state_pred, time_delta_pred

# --- 4. Instantiate and Train the Model ---

# Dummy data generation with the CORRECT shapes; replace with your real data loading
X_train = np.random.rand(TRAIN_SAMPLES, SEQUENCE_LENGTH, NUM_FEATURES).astype(np.float32)
y_job_id_train = np.random.randint(0, NUM_JOBS, TRAIN_SAMPLES).astype(np.int64)
y_state_train = np.random.randint(0, NUM_STATES, TRAIN_SAMPLES).astype(np.int64)
y_time_delta_train = np.random.rand(TRAIN_SAMPLES).astype(np.float32)

X_test = np.random.rand(TEST_SAMPLES, SEQUENCE_LENGTH, NUM_FEATURES).astype(np.float32)
y_job_id_test = np.random.randint(0, NUM_JOBS, TEST_SAMPLES).astype(np.int64)
y_state_test = np.random.randint(0, NUM_STATES, TEST_SAMPLES).astype(np.int64)
y_time_delta_test = np.random.rand(TEST_SAMPLES).astype(np.float32)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float().to(device)
y_job_id_train_tensor = torch.from_numpy(y_job_id_train).long().to(device)
y_state_train_tensor = torch.from_numpy(y_state_train).long().to(device)
y_time_delta_train_tensor = torch.from_numpy(y_time_delta_train).float().to(device)

X_test_tensor = torch.from_numpy(X_test).float().to(device)
y_job_id_test_tensor = torch.from_numpy(y_job_id_test).long().to(device)
y_state_test_tensor = torch.from_numpy(y_state_test).long().to(device)
y_time_delta_test_tensor = torch.from_numpy(y_time_delta_test).float().to(device)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_job_id_train_tensor, y_state_train_tensor, y_time_delta_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_job_id_test_tensor, y_state_test_tensor, y_time_delta_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize the model and move it to the device
model = GridSurrogate(NUM_FEATURES, D_MODEL, NUM_HEADS, NUM_LAYERS, FF_DIM, NUM_JOBS, NUM_STATES, DROPOUT_RATE).to(device)

print("Model Summary======================")
# Define loss functions and optimizer
criterion_job_id = nn.CrossEntropyLoss()
criterion_state = nn.CrossEntropyLoss()
criterion_time_delta = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    for batch_X, batch_y_job_id, batch_y_state, batch_y_time_delta in train_loader:
        optimizer.zero_grad()
        job_id_pred, state_pred, time_delta_pred = model(batch_X)

        # Calculate losses
        loss_job_id = criterion_job_id(job_id_pred, batch_y_job_id)
        loss_state = criterion_state(state_pred, batch_y_state)
        loss_time_delta = criterion_time_delta(time_delta_pred.squeeze(), batch_y_time_delta)

        # Combine losses
        total_loss = loss_job_id + loss_state + loss_time_delta
        total_loss.backward()
        optimizer.step()

    # Evaluation loop
    model.eval()
    with torch.no_grad():
        test_losses = []
        for batch_X, batch_y_job_id, batch_y_state, batch_y_time_delta in test_loader:
            job_id_pred, state_pred, time_delta_pred = model(batch_X)
            loss_job_id = criterion_job_id(job_id_pred, batch_y_job_id)
            loss_state = criterion_state(state_pred, batch_y_state)
            loss_time_delta = criterion_time_delta(time_delta_pred.squeeze(), batch_y_time_delta)
            test_losses.append([loss_job_id.item(), loss_state.item(), loss_time_delta.item()])

        avg_test_loss = np.mean(test_losses, axis=0)
        print(
            f"Epoch {epoch+1}/{EPOCHS}, "
            f"Train Loss: {total_loss.item():.4f}, "
            f"Test Job ID Loss: {avg_test_loss[0]:.4f}, "
            f"Test State Loss: {avg_test_loss[1]:.4f}, "
            f"Test Time Delta Loss: {avg_test_loss[2]:.4f}"
        )

# Save the trained model
torch.save(model.state_dict(), 'transformer_surrogate_model.pth')
print("Model training complete and saved.")


Using device: cpu
Model Summary======================
Epoch 1/50, Train Loss: 9.8571, Test Job ID Loss: 8.4538, Test State Loss: 1.3870, Test Time Delta Loss: 0.0877
Epoch 2/50, Train Loss: 10.1629, Test Job ID Loss: 8.4604, Test State Loss: 1.3880, Test Time Delta Loss: 0.0896
Epoch 3/50, Train Loss: 9.6628, Test Job ID Loss: 8.4854, Test State Loss: 1.3861, Test Time Delta Loss: 0.0855
Epoch 4/50, Train Loss: 9.9833, Test Job ID Loss: 8.5318, Test State Loss: 1.3864, Test Time Delta Loss: 0.0856
Epoch 5/50, Train Loss: 9.3796, Test Job ID Loss: 8.5360, Test State Loss: 1.3895, Test Time Delta Loss: 0.0856
Epoch 6/50, Train Loss: 9.6878, Test Job ID Loss: 8.5521, Test State Loss: 1.3866, Test Time Delta Loss: 0.0860
Epoch 7/50, Train Loss: 9.3409, Test Job ID Loss: 8.5800, Test State Loss: 1.3883, Test Time Delta Loss: 0.0861
Epoch 8/50, Train Loss: 9.4461, Test Job ID Loss: 8.5498, Test State Loss: 1.3872, Test Time Delta Loss: 0.0853
Epoch 9/50, Train Loss: 9.5664, Test Job ID Loss:

In [80]:
!pip install torchsummary

Defaulting to user installation because normal site-packages is not writeable


In [75]:
# Save the processed data and targets
processed_features_df.to_csv('processed_features.csv', index=False)

# Convert numpy arrays to pandas DataFrames and save as CSV
y_job_id_df = pd.DataFrame(y_job_id, columns=['ENCODED_JOB_ID'])
y_job_id_df.to_csv('y_job_id.csv', index=False)

y_state_df = pd.DataFrame(y_state, columns=['ENCODED_STATE'])
y_state_df.to_csv('y_state.csv', index=False)

y_time_delta_df = pd.DataFrame(y_time_delta, columns=['TIME_DELTA'])
y_time_delta_df.to_csv('y_time_delta.csv', index=False)

print("Processed data and targets saved successfully as CSV files.")

Processed data and targets saved successfully as CSV files.


In [76]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# --- 1. Define Model Parameters based on your data dimensions ---
# Use the exact dimensions from your preprocessing step
SEQUENCE_LENGTH = 10
NUM_FEATURES = 127
# These need to be derived from your preprocessed data (y_job_id, y_state)
# Example: NUM_JOBS = np.unique(y_job_id).size
# Example: NUM_STATES = np.unique(y_state).size
# You need to run the preprocessing script and capture these values.
NUM_JOBS = 4000  # Placeholder: Replace with the actual number of unique job IDs
NUM_STATES = 4   # Placeholder: Replace with the actual number of unique states

# Hyperparameters for the Transformer
D_MODEL = 64
NUM_HEADS = 4
NUM_LAYERS = 2
FF_DIM = 128
DROPOUT_RATE = 0.1
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 2. Define Positional Encoding Layer ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

# --- 3. Define the Multi-Task Transformer Model ---
class GridSurrogate(nn.Module):
    def __init__(self, num_features, d_model, num_heads, num_layers, ff_dim, num_jobs, num_states, dropout_rate=0.1):
        super(GridSurrogate, self).__init__()

        # Initial projection of features to the model dimension
        self.input_projection = nn.Linear(num_features, d_model)

        # Positional encoding to add sequence order information
        self.pos_encoder = PositionalEncoding(d_model)

        # PyTorch's built-in transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=ff_dim, dropout=dropout_rate, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Global average pooling to get a single vector representation
        self.avg_pool = nn.AdaptiveAvgPool1d(1)

        # Output heads for each task
        self.job_id_head = nn.Linear(d_model, num_jobs)
        self.state_head = nn.Linear(d_model, num_states)
        self.time_delta_head = nn.Linear(d_model, 1)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, num_features)

        # Project input features
        x = self.input_projection(x)

        # Add positional encoding
        x = self.pos_encoder(x)

        # Pass through the transformer encoder
        x = self.transformer_encoder(x)

        # Average pooling to get a single context vector
        x_pooled = self.avg_pool(x.transpose(1, 2)).squeeze(2)

        # Make predictions with each head
        job_id_pred = self.job_id_head(x_pooled)
        state_pred = self.state_head(x_pooled)
        time_delta_pred = self.time_delta_head(x_pooled)

        return job_id_pred, state_pred, time_delta_pred

# --- 4. Instantiate and Train the Model ---

# Dummy data generation to make the code runnable
# In a real scenario, you would load your actual preprocessed data here
X_train = np.random.rand(10023, 10, 17).astype(np.float32)
y_job_id_train = np.random.randint(0, NUM_JOBS, 8736).astype(np.int64)
y_state_train = np.random.randint(0, NUM_STATES, 8736).astype(np.int64)
y_time_delta_train = np.random.rand(8736).astype(np.float32)

X_test = np.random.rand(2185, 10, 17).astype(np.float32)
y_job_id_test = np.random.randint(0, NUM_JOBS, 2185).astype(np.int64)
y_state_test = np.random.randint(0, NUM_STATES, 2185).astype(np.int64)
y_time_delta_test = np.random.rand(2185).astype(np.float32)


# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float().to(device)
y_job_id_train_tensor = torch.from_numpy(y_job_id_train).long().to(device)
y_state_train_tensor = torch.from_numpy(y_state_train).long().to(device)
y_time_delta_train_tensor = torch.from_numpy(y_time_delta_train).float().to(device)

X_test_tensor = torch.from_numpy(X_test).float().to(device)
y_job_id_test_tensor = torch.from_numpy(y_job_id_test).long().to(device)
y_state_test_tensor = torch.from_numpy(y_state_test).long().to(device)
y_time_delta_test_tensor = torch.from_numpy(y_time_delta_test).float().to(device)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_job_id_train_tensor, y_state_train_tensor, y_time_delta_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_job_id_test_tensor, y_state_test_tensor, y_time_delta_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize the model and move it to the device
model = GridSurrogate(NUM_FEATURES, D_MODEL, NUM_HEADS, NUM_LAYERS, FF_DIM, NUM_JOBS, NUM_STATES, DROPOUT_RATE).to(device)

# Define loss functions and optimizer
criterion_job_id = nn.CrossEntropyLoss()
criterion_state = nn.CrossEntropyLoss()
criterion_time_delta = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    for batch_X, batch_y_job_id, batch_y_state, batch_y_time_delta in train_loader:
        optimizer.zero_grad()

        job_id_pred, state_pred, time_delta_pred = model(batch_X)

        # Calculate losses
        loss_job_id = criterion_job_id(job_id_pred, batch_y_job_id)
        loss_state = criterion_state(state_pred, batch_y_state)
        loss_time_delta = criterion_time_delta(time_delta_pred.squeeze(), batch_y_time_delta)

        # Combine losses (you can adjust weights here if needed)
        total_loss = loss_job_id + loss_state + loss_time_delta

        total_loss.backward()
        optimizer.step()

    # Evaluation loop
    model.eval()
    with torch.no_grad():
        test_losses = []
        for batch_X, batch_y_job_id, batch_y_state, batch_y_time_delta in test_loader:
            job_id_pred, state_pred, time_delta_pred = model(batch_X)

            loss_job_id = criterion_job_id(job_id_pred, batch_y_job_id)
            loss_state = criterion_state(state_pred, batch_y_state)
            loss_time_delta = criterion_time_delta(time_delta_pred.squeeze(), batch_y_time_delta)

            test_losses.append([loss_job_id.item(), loss_state.item(), loss_time_delta.item()])

        avg_test_loss = np.mean(test_losses, axis=0)
        print(f"Epoch {epoch+1}/{EPOCHS}, "
              f"Train Loss: {total_loss.item():.4f}, "
              f"Test Job ID Loss: {avg_test_loss[0]:.4f}, "
              f"Test State Loss: {avg_test_loss[1]:.4f}, "
              f"Test Time Delta Loss: {avg_test_loss[2]:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'transformer_surrogate_model.pth')
print("Model training complete and saved.")

Using device: cpu


AssertionError: Size mismatch between tensors